In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib import dates as mpl_dates
import re

In [19]:
#use df as basis to start all exploration
data = pd.read_csv(r"C:\Users\letru\Documents\Flatiron\Phase-3-Project\dsc-phase-3-project/Terry-Ohio_Stops.csv", parse_dates = ['date'],
                  index_col = 'date', dtype = {'officer_race':'category','officer_gender':'category',
                                                'subject_age':'category',
                                                'subject_race':'category','subject_gender': 'category',
                                                'stop_resolution': 'category','weapon_type':'category',
                                                'call_type':'category','arrest':'int32', 'frisk':'float',
                                                'precinct':'category', 'sector':'category', 'beat': 'category'})
data.sort_index(inplace = True)
data['count'] = 1
data

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat,count
date,,,,,,,,,,,,,,,,,,,,,
2015-03-15,1 - 17,-1,2.020000e+13,28020,Referred for Prosecution,Lethal Cutting Instrument,4585,1955,M,Hispanic or Latino,...,-,-,-,NaN,N,Y,East,G,G2,1
2015-03-16,18 - 25,-1,2.010000e+13,305901,Arrest,None,7661,1984,M,White,...,-,-,-,NaN,N,N,West,M,M3,1
2015-03-16,18 - 25,-1,2.020000e+13,28093,Field Contact,None,7634,1977,M,White,...,-,-,-,NaN,N,N,-,-,-,1
2015-03-16,36 - 45,-1,2.020000e+13,28092,Field Contact,None,7634,1977,M,White,...,-,-,-,NaN,N,N,-,-,-,1
2015-03-17,26 - 35,-1,2.020000e+13,28755,Offense Report,None,7429,1984,M,White,...,THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),#NAME?,911,NaN,N,N,East,G,G3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-18,46 - 55,7739808507,2.020000e+13,25603848346,Arrest,-,7722,1989,M,White,...,FIGHT - IP - PHYSICAL (NO WEAPONS),#NAME?,911,WEST PCT 2ND W - MARY,Y,N,West,D,D2,1
2021-06-18,36 - 45,7727013190,2.020000e+13,25603571028,Arrest,-,7766,1984,M,White,...,ASLT - WITH OR W/O WEAPONS (NO SHOOTINGS),#NAME?,"TELEPHONE OTHER, NOT 911",WEST PCT 2ND W - DAVID BEATS,Y,N,West,M,M1,1
2021-06-18,18 - 25,7728717651,2.020000e+13,25603965158,Arrest,-,7722,1989,M,White,...,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),#NAME?,ONVIEW,WEST PCT 2ND W - MARY,Y,N,West,M,M2,1


In [20]:
new_df = data.copy()
new_df.reset_index(inplace=True)
new_df

,date,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,...,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat,count
0,2015-03-15,1 - 17,-1,2.020000e+13,28020,Referred for Prosecution,Lethal Cutting Instrument,4585,1955,M,...,-,-,-,NaN,N,Y,East,G,G2,1
1,2015-03-16,18 - 25,-1,2.010000e+13,305901,Arrest,None,7661,1984,M,...,-,-,-,NaN,N,N,West,M,M3,1
2,2015-03-16,18 - 25,-1,2.020000e+13,28093,Field Contact,None,7634,1977,M,...,-,-,-,NaN,N,N,-,-,-,1
3,2015-03-16,36 - 45,-1,2.020000e+13,28092,Field Contact,None,7634,1977,M,...,-,-,-,NaN,N,N,-,-,-,1
4,2015-03-17,26 - 35,-1,2.020000e+13,28755,Offense Report,None,7429,1984,M,...,THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),#NAME?,911,NaN,N,N,East,G,G3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47991,2021-06-18,46 - 55,7739808507,2.020000e+13,25603848346,Arrest,-,7722,1989,M,...,FIGHT - IP - PHYSICAL (NO WEAPONS),#NAME?,911,WEST PCT 2ND W - MARY,Y,N,West,D,D2,1
47992,2021-06-18,36 - 45,7727013190,2.020000e+13,25603571028,Arrest,-,7766,1984,M,...,ASLT - WITH OR W/O WEAPONS (NO SHOOTINGS),#NAME?,"TELEPHONE OTHER, NOT 911",WEST PCT 2ND W - DAVID BEATS,Y,N,West,M,M1,1
47993,2021-06-18,18 - 25,7728717651,2.020000e+13,25603965158,Arrest,-,7722,1989,M,...,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),#NAME?,ONVIEW,WEST PCT 2ND W - MARY,Y,N,West,M,M2,1
47994,2021-06-18,36 - 45,7730559318,2.020000e+13,25602140356,Field Contact,-,8471,1992,M,...,"WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)",#NAME?,911,WEST PCT 1ST W - KING/QUEEN,N,Y,West,Q,Q3,1


In [21]:
new_df = new_df[['Subject Age Group', 'Subject ID', 'Stop Resolution',
         'Weapon Type', 'Officer ID', 'Officer YOB', 
         'Officer Gender', 'Officer Race', 'Subject Perceived Race', 
         'Subject Perceived Gender', 'date', 'Initial Call Type','Call Type',
         'Officer Squad', 'Arrest Flag', 'Frisk Flag',
         'Precinct', 'Sector', 'Beat' ]]

In [22]:
new_df.columns = ['subject_age', 'subject_id', 'stop_resolution',
         'weapon_type', 'officer_id', 'officer_age', 
         'officer_gender', 'officer_race', 'subject_race',
         'subject_gender', 'date', 'initial_call_type', 'call_type',
         'officer_squad', 'arrest', 'frisk',
         'precinct', 'sector', 'beat']

In [23]:
new_df = new_df.astype({'subject_age': 'category', 'officer_gender':'category', 
                'officer_race': 'category', 'subject_race':'category', 
                'subject_gender':'category', 'date':'datetime64','arrest':'category', 
                'frisk':'category','precinct':'category', 'sector':'category',
                'beat': 'category'})
new_df.replace('-', np.nan, inplace = True)

In [24]:
def split_mean(x):
    #Function to split the Age bins and return the mean of the two numbers
        if '-' in x:
            split_list = x.split('-')
            mean = (float(split_list[0]) + float(split_list[1]))/2
        else:
            mean = 56
        return mean

In [25]:
columns_to_drop = ['officer_squad','stop_resolution','officer_race', 'officer_age', 'officer_gender', 'weapon_type', 'arrest','initial_call_type','precinct', 'sector','subject_id']

In [32]:
df = new_df.drop(columns_to_drop, axis = 1)

df.head()

,subject_age,officer_id,subject_race,subject_gender,date,call_type,frisk,beat
0,1 - 17,4585,Black or African American,Female,2015-03-15,NaN,Y,G2
1,18 - 25,7661,Black or African American,Male,2015-03-16,NaN,N,M3
2,18 - 25,7634,White,Male,2015-03-16,NaN,N,NaN
3,36 - 45,7634,Multi-Racial,Male,2015-03-16,NaN,N,NaN
4,26 - 35,7429,Black or African American,Male,2015-03-17,911,N,G3


In [39]:
#replace age bins with the mean for the subject_age column
# removing the 'Unknown' & 'Other' entries for the target variable

df = df[(df.subject_race != 'Unknown') & (df.subject_race != 'Other')]

#subset only the call type of 'onview'
df = df[df.call_type == 'ONVIEW']

#drop NaN values in the dataframe
df.dropna(inplace = True)

#remove all unused categories
for col in df.select_dtypes(include = ['category']).columns:
    df[col] = df[col].cat.remove_unused_categories()

#create a dictionary of the category codes and corresponding race strings
race_cat_codes = dict(enumerate(df['subject_race'].cat.categories))  
    
#create the pattern to match the beat entries
pattern = re.compile( '^[A-Z][1-9]$')

#drop any NaNs in the beat column
df = df.dropna(subset = ['beat'])

#use the pattern created to subset the data and get rid of the erroneous entries
df = df[(df.beat.str.contains(pattern))]

#remove all unused categories
for col in df.select_dtypes(include = ['category']).columns:
    df[col] = df[col].cat.remove_unused_categories()

    

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1619 entries, 480 to 47987
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_age     1619 non-null   category      
 1   officer_id      1619 non-null   object        
 2   subject_race    1619 non-null   category      
 3   subject_gender  1619 non-null   category      
 4   date            1619 non-null   datetime64[ns]
 5   call_type       1619 non-null   object        
 6   frisk           1619 non-null   category      
 7   beat            1619 non-null   category      
dtypes: category(5), datetime64[ns](1), object(2)
memory usage: 62.0+ KB


In [37]:
#find the value counts of each officer, and create a mask for 10 or more occurences
#create a list of officers that have 10 or more frisks
ls = list(df.officer_id.value_counts()[df.officer_id.value_counts() >= 10].index)

#use the above list to filter the data for only officers with 10 or more frisks
df = df[df.officer_id.isin(ls)]

df.head()

,subject_age,officer_id,subject_race,subject_gender,date,call_type,frisk,beat
480,30.5,7428,Black or African American,Male,2015-05-21,ONVIEW,Y,K3
491,40.5,7428,Asian,Male,2015-05-21,ONVIEW,N,K3
493,40.5,7428,Asian,Male,2015-05-21,ONVIEW,N,K3
496,40.5,7428,Black or African American,Female,2015-05-21,ONVIEW,Y,K3
9525,56.0,8302,Black or African American,Male,2016-04-18,ONVIEW,N,K2


In [38]:
df.frisk.value_counts()

N    1403
Y     216
Name: frisk, dtype: int64

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

#import preprocessing, metrics & pipelines
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, precision_score

ModuleNotFoundError: No module named 'imblearn'